<a href="https://colab.research.google.com/github/pasquale90/mthesis/blob/master/code/2.flat_UrbanSound8k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall librosa
!pip install librosa

In [ ]:
#prevent from disconnecting --> to console
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}setInterval(ClickConnect,60000)
'''

In [ ]:
#Import Google_drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#Import libraries
import os
import pandas as pd
import librosa
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook as tqdm
import math
from torch.autograd import Variable
from torchvision import transforms, models
from google.colab import files as filez

In [ ]:
#define device
if torch.cuda.is_available():
  device=torch.device('cuda:0')
else:
  device=torch.device('cpu')
print(device)

cuda:0


In [ ]:
#define experiment mode
class mode_class:
  def __init__(self, mode_atr):
    if (mode_atr == 80):
      self.mode='80'
    elif (mode_atr == 128):
      self.mode = '128'
    elif (mode_atr == 360):
      self.mode = '360'
    else:
      print(f'{mode_atr} input attribute is not valid.Please insert 80 or 128 or even 360')
  def get_mode(self):
    return self.mode

mode_id = 80
#mode_id = 128
#mode_id = 360

#define mode
mode_instance = mode_class(mode_id)
mode=mode_instance.get_mode()
print(mode)

80


In [ ]:
#define expid
expid='us2'

#define paths
data_path='/content/gdrive/My Drive/dissertation/UrbanSound8K/metadata/UrbanSound8K.csv'
audio_path='/content/gdrive/My Drive/dissertation/UrbanSound8K/audio/'

#model.py		
model_savepath= "/content/"+expid+mode+"/saved_models/"

#store.py
results_path="/content/"+expid+mode+"/results/"
compare_results_path ="/content/"+expid+mode+"/k_fold_results/"	

#attempt.py
attempt_path="/content/"+expid+mode+"/expattempt/"

In [ ]:
#read us8k folds method
def read_folds(audio_path):
  folds=os.listdir(audio_path)
  
  order={}
  files={}
  filesum=0
  
  for f in folds:
  
    filelist = os.listdir(audio_path+f+'/')
   
    if (f[-1:]!='0'):
      num=f[-1:]
    elif(f[-1:]=='0'):#get the value of 10 instead of 0
      num=f[-2:]
  
    order[f]=int(num)
    files[f]=filelist

    filesum+=len(files[f])
  
  #folds ordered
  folds={}
  for key, value in sorted(order.items(), key=lambda item: item[1]):
    folds[value]= key
  folds=list(folds.values())
  
  return folds, files, order, filesum

In [ ]:
#Import Dataset
us8k = pd.read_csv(data_path)
print(us8k.shape)

folds, audiofiles, order, filesum = read_folds(audio_path)
for f in folds:
    print('fold no_%d contains %d audiofiles'%(order[f],len(audiofiles[f]))) 
print('All in all there are %d audio files found in 8k Urban Sound dataset folders'%filesum)

print(us8k.columns)
us8k.rename(columns={'class':'Class'}, inplace=True)
print('\n\ncolumn <class> became... <%s>'%us8k.columns[-1])

#store_sorted_classes in the same way as defined in data.py
us8k_classes = sorted(us8k['Class'].unique())
num_classes = len(us8k_classes)
print('num_classes: ',num_classes)

In [ ]:
#AUDIO AUGMENTATION FUNCTIONS_ synthetic data
def audio_augmentation(data, sr, class_conditional, shift_time, thresshold=0.5):

  #add_white_noise to the signal
  def white_noise(data): 
    noiz = np.random.randn(len(data))
    mean_intensity = np.sum(np.square(data))/len(data)
    data_wn = data + noiz*0.75* mean_intensity
    return data_wn
    
  #Shift the sound wave by a factor value chosen randomly within [0.5,1,1.5,2] seconds
  def time_shift(data,sr):
    time_step = np.random.choice([sr//2,sr,sr+sr//2,sr*2])
    time_shifted = np.roll(data,time_step)
    return time_shifted

  #Time-stretching the wave by a factor value of 0.9. Permissible : 0 < x < 1.0
  def time_stretch(data):#,factor
    factor = 0.90
    time_streched = librosa.effects.time_stretch(data,factor)
    return time_streched[0:len(data)]

  #pitch shifting of wave by a random factor value in the space [-1,1].  Permissible : -5 <= x <= 5
  def pitch_shift(data,sr):
    detune = np.random.uniform(low=-2.5,high=-1.75,size=None)
    overtune = np.random.uniform(low=1.75,high=2.5,size=None)
    shift_factor = np.random.choice([detune,overtune])
    pitch_shifted = librosa.effects.pitch_shift(data,sr,n_steps=shift_factor)
    return pitch_shifted
  
  def soft_pitch_shift(data,sr):
    detune = np.random.uniform(low=-1.0,high=-0.5,size=None)
    overtune = np.random.uniform(low=0.5,high=1.0,size=None)
    shift_factor = np.random.choice([detune,overtune])
    pitch_shifted = librosa.effects.pitch_shift(data,sr,n_steps=shift_factor)
    return pitch_shifted

  '''
  #A rough but very simple segmentation mask function
  def envelope(y,rate,threshold):
    mask = []
    y= pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10), min_periods=1, center=True).mean()#/10 means that win=1/10 sec
    for mean in y_mean:
      if mean > threshold:#if above the threshold keep
        mask.append(True)
      else:
        mask.append(False)#else drop
    return mask
  '''
  
  strong_augs = ['airplane','car_horn','cat',#esc
                 'chirping_birds','church_bells',
                 'cow','crow','crying_baby',
                 'door_wood_creaks','insects',
                 'rooster','sheep','siren',
                 'car_horn','children_playing','siren']#us8k

  medium_augs= ['breathing','brushing_teeth',#esc
                'clock_alarm','coughing','dog',
                'door_wood_knock','fireworks',
                'frog','glass_breaking','hand_saw',
                'hen','laughing','pig','pouring_water',
                'sneezing','snoring',
                'dog_bark','gun_shot','street_music']#us8k
                
  weak_augs = ['can_opening','chainsaw','clapping',#esc
               'clock_tick','crackling_fire','crickets',
               'drinking_sipping','engine','footsteps',
               'helicopter','keyboard_typing','mouse_click',
               'rain','sea_waves','thunderstorm',
              'toilet_flush','train','vacuum_cleaner',
              'washing_machine','water_drops','wind',  
               'air_conditioner','drilling','engine_idling','jackhammer']#us8k
   
  #prob_wn = np.random.uniform(low=0,high=1)
  #if prob_wn>thresshold:
  data =  white_noise(data)
  
  #prob_tsh = np.random.uniform(low=0,high=1)
  #if prob_tsh>thresshold:

  # if it is padded , don 't shift
  if shift_time: 
    data = time_shift(data,sr)

  if class_conditional in strong_augs:
    
    prob_tst = np.random.uniform(low=0,high=1)
    if prob_tst>thresshold:
      data = time_stretch(data)

    prob_psh = np.random.uniform(low=0,high=1)
    if prob_psh>thresshold:
      data = pitch_shift(data,sr)
  
  elif class_conditional in medium_augs:
    
    prob_spsh = np.random.uniform(low=0,high=1)
    if prob_spsh>thresshold:
      data = soft_pitch_shift(data,sr)

  elif class_conditional in weak_augs:
    pass

  return data

#Νο Vision Augmentations in this exp
'''
train_transforms = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
        transforms.RandomErasing(p=0.5, scale = (0.05,0.05), ratio = (0.3,0.33), value=0, inplace=False)      
    ])
valid_transforms = transforms.Compose([
        transforms.ToTensor()
    ])
'''

'\ntrain_transforms = transforms.Compose([\n        transforms.RandomHorizontalFlip(),\n        transforms.ToTensor()\n        transforms.RandomErasing(p=0.5, scale = (0.05,0.05), ratio = (0.3,0.33), value=0, inplace=False)      \n    ])\nvalid_transforms = transforms.Compose([\n        transforms.ToTensor()\n    ])\n'

In [ ]:
#define analysis parameters
def analysis_parameters(mode):
  sampling_rate=44100
  hop_length=512
  fft_points=2048
  mel_bands=mode  #80x321 or 128x321
  return sampling_rate, hop_length, fft_points, mel_bands
  
def zero_pad(signal,fs):
  shift_time = True
  if len(signal)>(4*fs):
    signal = signal[0:4*fs]
  elif len(signal)<(4*fs):
    shift_time = False
  num_zeros=4*fs-len(signal)
  zp=np.zeros(num_zeros,dtype=float)
  padded_signal = np.concatenate((signal,zp),axis=0)
  return padded_signal, shift_time

#extract features
def extract_mel_spectogram(audio_path, df, folds, audiofiles, sr, hop, nfft, nmels):   

  def compute_mel_spectogram(raw,sr,hop,nfft,nmels,window='hann'):
    S=librosa.feature.melspectrogram(y=raw.astype(float),
                                      sr=sr,S=None,
                                      n_fft=nfft,
                                      hop_length=hop,
                                      window=window, 
                                      power=2,
                                      n_mels=nmels) 
    S = librosa.power_to_db(S, ref=np.max)
    return S

  def scale_image(spec, eps=1e-6):
    mean = spec.mean()
    std = spec.std()
    spec_norm = (spec - mean) / (std + eps)
    spec_min, spec_max = spec_norm.min(), spec_norm.max()
    spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
    spec_scaled = spec_scaled.astype(np.uint8)
    return spec_scaled

  def flatten_features(feature):
    feature = feature.T.flatten()
    return feature

  features, labels, folders = [], [], []  
 
  extr = True
  if extr == True:

    print('Extracting features ........ ')
    
    shape_print = True
    pad_print = True

    #deterministic random augmentation
    np.random.seed(77)

    for foldname, filesinfold in audiofiles.items():
      
     #if foldname=='fold9' or foldname=='fold10':#test

      path = audio_path+foldname+'/'

      for file in tqdm(filesinfold):						#[0:200]: test
        name = file.split('.wav')[0]
        label = np.int8(file.split('-')[1])
        folder = np.int8(folds.index(foldname)+1)

        raw,_ = librosa.load(path+file, sr=sr, mono=True)
        #zero pad signal to 5 seconds
        padded, shift_time = zero_pad(raw,sr)
        #extract mel spectogram
        S = compute_mel_spectogram(padded,sr,hop,nfft,nmels)
        #flip image
        flipped = np.flipud(S)
        #to gray scale
        greyscale = scale_image(flipped)
        #flatten features
        flattened = flatten_features(greyscale)
      
        features.append(flattened)
        labels.append(label)
        folders.append(folder)

        #Synthetic data augmentations
        prob = np.random.uniform(low=0,high=1)
        if prob<=1:#100% of the files
          category = df.loc[df['slice_file_name']==file]['Class'].to_string(index=False).lstrip()
          augmented = audio_augmentation(data=padded,sr=sr,class_conditional=category,shift_time=shift_time,thresshold=0.5)
          synthetic = scale_image(np.flipud(compute_mel_spectogram(augmented,sr,hop,nfft,nmels)))
          flatsynth = flatten_features(synthetic)

          features.append(flatsynth)
          labels.append(label)
          folders.append(folder)

        #test_shapes of raw data and feature representation
        if shape_print:
          print('\nFeature Shape Check\n')
          print(f'raw had len:{len(raw)/sr}, and padded has len:{len(padded)/sr}')
          print(f'Spectogram has shape : {flattened.shape} with min:{flattened.min()} and max:{flattened.max()}]')
          shape_print = False
        if (not shift_time) and pad_print:
          print('\nPadded Feature Shape Check\n')
          print(f'raw had len:{len(raw)/sr}, and padded has len:{len(padded)/sr}')
          print(f'Spectogram has shape : {flatsynth.shape} with min:{flatsynth.min()} and max:{flatsynth.max()}')
          pad_print = False

  '''
  print('len(features)-features',len(features))
  print('len(features[0])-freq_domain',len(features[0]))
  print('labels',len(labels))
  print('folders',len(folders))
  '''
  
  print('Features are extracted!')

  return features, labels, folders

In [ ]:
#get analysis parameters
sampling_rate, hop_length, fft_points, mel_bands = analysis_parameters(mode)
print(f'sampling_rate: {sampling_rate}, hop_length: {hop_length}, fft_points: {fft_points}, mel_bands: {mel_bands}')

#extract_features
features, labels, folders = extract_mel_spectogram(audio_path,us8k,folds,audiofiles,sampling_rate,hop_length,fft_points,mel_bands)
print(f' feature\'s len : {len(features)}, labels : {len(labels)}, folders : {len(folders)}')

In [ ]:
#train and fold definition
vfold = 10
train_folds = [folds.index(fold)+1 for i,fold in enumerate(folds) if vfold!=folds.index(fold)+1]
#train_folds = [folds.index(fold)+1 for i,fold in enumerate(folds) if (vfold-1)==folds.index(fold)+1]#for testing
valid_folds = [folds.index(fold)+1 for i,fold in enumerate(folds) if vfold==folds.index(fold)+1]
print('train_folds: ',train_folds)
print('valid_folds: ',valid_folds)

In [ ]:
class Data(Dataset): 
  def __init__(self, features,labels,folders,split):
    
    def convert_to_tensor(data,transform):
      tensors = []
      for file in data:
        tensor = torch.Tensor(list(file))
        tensors.append(tensor)
      return tensors

    print('Loading_features......')

    #features, labels, folders
    self.indexes = [i for i, val in enumerate(folders) if val in split]
    self.data = [features[x] for x in self.indexes]
    self.labels = [labels[x] for x in self.indexes]

    #normalize
    self.data = np.asarray(self.data, dtype=np.float32)/255.0
    print(type(self.data))
    print(self.data.shape)

    #convert numpy to tensor
    self.data = convert_to_tensor(self.data,transforms)

  def __len__(self):
    return len(self.data)
  def __getitem__(self, idx):#load data on demand
    return self.data[idx], self.labels[idx]

  

#load features
train_data = Data(features,labels,folders,train_folds)
valid_data = Data(features,labels,folders,valid_folds)
print('features are loaded')

Loading_features......
<class 'numpy.ndarray'>
(15790, 27600)
Loading_features......
<class 'numpy.ndarray'>
(1674, 27600)
features are loaded


In [ ]:
batch_size = 16	
print(f'batch_size: {batch_size}')

#data iterator
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)

In [ ]:
class Fcnn7(nn.Module):
    def __init__(self, num_cats):
        super(Fcnn7, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU())
        self.conv2 = nn.Sequential(
            nn.Conv1d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(4, stride=3))
        self.conv3 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU())
        self.conv4 = nn.Sequential(
            nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(4, stride=3))
        self.conv5 = nn.Sequential(
            nn.Conv1d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU())
        self.conv6 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(4,stride=3))
        self.conv7 = nn.Sequential(
            nn.Conv1d(256, 512, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm1d(512),
            nn.ReLU())
        self.conv8 = nn.Sequential(
            nn.Conv1d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.MaxPool1d(4,stride=3))
        self.ap = nn.AdaptiveMaxPool1d(4)
        self.fc = nn.Linear(512*4,512) 
        self.dropout = nn.Dropout(0.25)
        self.fc2 = nn.Linear(512,512)
        self.fc3 = nn.Linear(512,num_cats)
    
    def forward(self, x):
        x = x.view(x.shape[0], 1,-1 )
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x) 
        x = self.conv6(x)        
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.ap(x)
        x = x.view(x.shape[0], x.size(1) * x.size(2))
        x = self.dropout(F.relu(self.fc(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [ ]:
#introduce reproducibility
seed = 700
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#init model
#model =  Fcnn1(num_cats=num_classes).cuda()#to(device)
model =  Fcnn7(num_cats=num_classes).to(device)
modelid='Fcnn7'
total_params = sum(p.numel() for p in model.parameters())
print(f'model_{modelid} initialized with total : {total_params} parameters.')

model_Fcnn7 initialized with total : 2883402 parameters.


In [ ]:
#define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-5
epochs = 34#30
print(f' learning_rate = {learning_rate}, epochs = {epochs}')
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
#define metrics
def F1_score(trace_y, trace_yhat, classes):
  num_classes = len(classes)
  #confussion matrix
  confmat = confusion_matrix(trace_y, trace_yhat, num_classes)
    
  TP=pd.Series(confmat.to_numpy().diagonal())
  FP=confmat.sum(axis=0)-TP
  FN=confmat.sum(axis=1)-TP

  TN=confmat.sum().sum()-TP-FP-FN
  
  #micro
  micro_precision = TP.sum()/(TP.sum()+FP.sum())
  micro_recall = TP.sum()/(TP.sum()+FN.sum())
  F1_micro=2*micro_precision*micro_recall/(micro_precision+micro_recall)

  #macro
  macro_precision = pd.Series(np.nan)
  macro_recall= pd.Series(np.nan)
  macro_f1 = pd.Series(np.nan)
  #Avoid Zero-Division
  for i in range(num_classes):
    if (TP[i]==0 and FP[i]==0):
      macro_precision[i]=0
    else:
      macro_precision[i] = TP[i]/(TP[i]+FP[i]) 
    if (TP[i]==0 and FN[i]==0):
      macro_recall[i]=0
    else:
      macro_recall[i] = TP[i]/(TP[i]+FN[i])
    
    if (macro_precision[i]+macro_recall[i]==0.0):
      macro_f1[i]=0
    else:
      macro_f1[i] = 2*macro_precision[i]*macro_recall[i]/(macro_precision[i]+macro_recall[i])
      
  macro_precision = macro_precision.sum()/num_classes
  macro_recall=macro_recall.sum()/num_classes
  F1_macro = macro_f1.sum()/num_classes

  return micro_recall,micro_precision,F1_micro, macro_recall,macro_precision,F1_macro
  
#labels, preds
def confusion_matrix(trace_y,trace_yhat, num_classes):
  confmat=pd.DataFrame(data=np.zeros(shape=(num_classes,num_classes)))
  predictions=trace_yhat.argmax(axis=1)

  for i,pred in enumerate(predictions):
    confmat.iat[trace_y[i],pred]+=1

  return confmat


def F1_Class(trace_y,trace_yhat,classes):
  num_classes = len(classes)
  confmat = confusion_matrix(trace_y, trace_yhat, num_classes)
  
  TP=pd.Series(confmat.to_numpy().diagonal())
  FP=confmat.sum(axis=0)-TP
  FN=confmat.sum(axis=1)-TP
  TN=confmat.sum().sum()-TP-FP-FN

  #AVOID ZERO DIVISION
  class_precision = pd.Series(np.nan)
  class_recall = pd.Series(np.nan)
  class_f1 = pd.Series(np.nan)
  for i in range(num_classes):
    if (TP[i]==0 and FP[i]==0):
      class_precision[i]=0
    else:
      class_precision[i] = TP[i]/(TP[i]+FP[i]) 
    if (TP[i]==0 and FN[i]==0):
      class_recall[i]=0
    else:
      class_recall[i] = TP[i]/(TP[i]+FN[i])
    
    if (class_precision[i]==0.0 and class_recall[i]==0.0):
      class_f1[i]=0
    else:
      class_f1[i] = 2*class_precision[i]*class_recall[i]/(class_precision[i]+class_recall[i])
    
  #create a dict_report
  f1_class_report = {}
  class_report = {}
  class_counts = np.asarray(np.unique(trace_y, return_counts=True)).T

  for i,c in enumerate(classes):
    f1_class_report[c] = {}
    f1_class_report[c]['precision'] = class_precision[i]
    f1_class_report[c]['recall'] = class_recall[i]
    f1_class_report[c]['f1'] = class_f1[i]
    f1_class_report[c]['count'] = class_counts[i][1]

  return f1_class_report


from sklearn.metrics import classification_report
import sklearn
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

def backup_metrics(labels,preds,classes,path):
    if (not os.path.exists(path)):
      os.makedirs(path)
    report = classification_report(labels, preds.argmax(1), target_names=classes)
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-5]:
        row = {}
        row_data = ' '.join(line.split())   
        row_data = row_data.split(' ')
        row['class'] = row_data[0]
        row['precision'] = float(row_data[1])
        row['recall'] = float(row_data[2])
        row['f1_score'] = float(row_data[3])
        row['support'] = float(row_data[4])
        report_data.append(row)
    dataframe = pd.DataFrame.from_dict(report_data)
    dataframe.to_csv(path+'backup_classification_report.csv',index=False)

    accuracy= f1_score(labels, preds.argmax(1), average='micro', zero_division='warn')
    macro_avg= f1_score(labels, preds.argmax(1), average='macro', zero_division='warn')
    weightedf1= f1_score(labels, preds.argmax(1), average='weighted', zero_division='warn')

    precision,recall,_,_ =precision_recall_fscore_support(labels, preds.argmax(1), average='macro')
    general = pd.DataFrame(data = [accuracy,recall,precision,macro_avg,weightedf1],index=['accuracy','recall','precision','macro_f1','weightedf1'])
    general.to_csv(path+'backup_general.csv')

In [ ]:
#prevent overfitting custom method
class prevent_overfitting:
  def __init__(self):
    self.tolerance=5  #how many epochs tolerance
    self.minloss = 10 
    self.lossenvelope =[]
    self.thresshold = 0.5
    self.avgperformances=[]
    self.best_epoch = 0
    self.micro_accuracies = []
    self.macro_accuracies = []
    self.best_curr_model = False#None #model.state_dict save
  
  #check overfitting by observing 3 last epoch's valid loss
  def detect_overfitting(self,validloss,epoch,console_path):
    self.lossenvelope.append(validloss)
    if validloss<self.minloss:
      self.minloss=validloss
    print('minloss',self.minloss)
    print('minloss',self.minloss,file=open(console_path, "a"))

    if (len(self.lossenvelope)>self.tolerance):
      self.lossenvelope.pop(0) #reject first value when {tolerance} values are passed

    if(len(self.lossenvelope)>=self.tolerance):
      overfit=all(earlier <= later for earlier, later in zip(self.lossenvelope, self.lossenvelope[-3:]))#check if descending
      if (overfit and min(self.lossenvelope)>self.minloss):
        return True
    return False
  
  #store best model's results by observing mean micro and macro accuracies
  def store_best_model(self,micro_accuracy,macro_accuracy,console_path):
    self.macro_accuracies.append(micro_accuracy)
    self.micro_accuracies.append(macro_accuracy)
    
    meanaccuracy = (micro_accuracy+macro_accuracy)/2.0
    self.avgperformances.append(meanaccuracy)
    
    self.best_epoch = self.avgperformances.index(max(self.avgperformances))+1
    
    if meanaccuracy >= self.thresshold:
      self.best_curr_model = (meanaccuracy >= max(self.avgperformances))
      print(f'just saved the best current model in epoch{self.best_epoch}, with acc1:{self.micro_accuracies[self.best_epoch-1]}, and acc2:{self.macro_accuracies[self.best_epoch-1]}')
      print(f'just saved the best current model in epoch{self.best_epoch}, with acc1:{self.micro_accuracies[self.best_epoch-1]}, and acc2:{self.macro_accuracies[self.best_epoch-1]}', file=open(console_path, "a"))
    else:
      self.best_curr_model = False

    return self.best_curr_model, self.micro_accuracies[self.best_epoch-1],self.macro_accuracies[self.best_epoch-1]#, self.best_epoch
  
  def early_stopping(self,epochinst):
    print('training is terminating so as to prevent further overfitting')
    total_epochs = epochinst.set_total(epochinst.get_step())
    return total_epochs

class epochs_class:
  def __init__(self):
    self.total_epochs=50#random value
    self.step_epoch=1
  def set_total(self,num_epochs):
    self.total_epochs=num_epochs
    return self.get_total()
  def get_total(self):
    return self.total_epochs
  def next_step(self):
    self.step_epoch+=1
    return self.step_epoch
  def get_step(self):
    return self.step_epoch

In [ ]:
#save results
def save_results(content, name):
  path = results_path
  if (not os.path.exists(path)):
    os.mkdir(path)
  ovr_results, class_results = content
  ovr_results_filename, class_results_filename = name 
  
  ovr_results.to_csv(path+ovr_results_filename) 
  class_results.to_csv(path + class_results_filename) 
   
#func to define the last argument in the save_results method
#PATTERN:dataset&exp_mode_validationfold_attempt_metrics.csv
def define_filenames_pattern(expid, mode, vfold, attempt):
  classf1_report_filename = expid+'_'+mode+'_v'+str(vfold)+'_a'+str(attempt)+'_classF1.csv'
  ovr_results_filename=expid+'_'+mode+'_v'+str(vfold)+'_a'+str(attempt)+'_overalF1.csv'
  return ovr_results_filename , classf1_report_filename
  
  
#as the first argument in save_results
def define_content(epochs,
		               mean_train_losses, 
                   mean_valid_losses, 
                   microrecall,
                   microprecision,
                   microf1, 
                   macrorecall,
                   macroprecision,
                   macrof1, 
		               classf1):
                   
  #data = [mean_train_losses, mean_valid_losses, accuracies,micro_auroc, macro_auroc microf1, macrof1]#auroc,
  data = {'mean_train_loss' : mean_train_losses, 'mean_valid_loss' : mean_valid_losses, 
  'micro_recall':microrecall, 'micro_precision': microprecision, 'micro_f1' : microf1,
  'macro_recall':macrorecall, 'macro_precision': macroprecision, 'macro_f1' : macrof1}
  
  #index names
  epochs_index= (['epoch_'+str(ep+1) for ep in range(epochs)])
  
  overal_results = pd.DataFrame(data=data, index=epochs_index,  dtype=np.float16)
  class_results = pd.DataFrame(data=classf1, index=epochs_index,  dtype=np.float16)
  
  overal_results.index.name = 'epochs'
  class_results.index.name = 'epochs'
  
  return overal_results, class_results

 
#save general results so as to compare with other systems in a different folder
def save_genres(micro, macro, params, validation_fold, filename):
  
  path = compare_results_path

  #if csv exists, overwrite results
  if os.path.isfile(path+filename):
    general_results=pd.read_csv(path+filename)
    general_results.set_index('validation_fold:',inplace=True)
    #debug
    print(general_results.shape)
    new_fold_results = [micro, macro, params]
    #debug
    print(len(new_fold_results))
    general_results[validation_fold] = new_fold_results
    general_results.to_csv(path+filename)
  #if not, store them into a new csv file
  else:
    os.makedirs(path)
    data = {validation_fold : [micro,macro,params]}
    general_results = pd.DataFrame(data=data, index=['micro_f1','macro_f1','params'], dtype=np.float16)
    general_results.index.name = 'validation_fold:'
    general_results.to_csv(path+filename) 
    print(general_results)


def genres_filename(expid,mode):
  filename=expid+'_'+str(mode)+'.csv'
  return filename

#save the model
#PATTERN:exp-num_mode_dataset_'model'_attempt		
def save_model(model_name, vfold, state_dict):
  path = model_savepath
  if not os.path.exists(path):
    os.mkdir(path)
  torch.save(state_dict, path + model_name+'_'+str(vfold)+'.pt')
'''
#load model
def load_model(model_name, mode):
  if (mode=='80'):
    model = esc_mel_model_hybrid(input_shape=(1,80,431), batch_size=16, num_cats=50)
  elif(mode=='128'):
    model = esc_mel_model_hybrid(input_shape=(1,128,431), batch_size=16, num_cats=50)
  model.load_state_dict(torch.load(paths.model_savepath+model_name+'.pth'))
'''

def ZipAnDownload(mode):
  if mode == '80':
    !zip -r /content/us280.zip /content/us280
    filez.download("/content/us280.zip")
  elif mode == '128':
    !zip -r /content/us2128.zip /content/us2128
    filez.download("/content/us2128.zip")
  elif mode == '360':
    !zip -r /content/us2360.zip /content/us2360
    filez.download("/content/us2360.zip")

In [ ]:
class attempt_class:
  #init_attempt
  def __init__(self,mode,vfold):

    self.attempt_path = attempt_path
    self.attempt_file = attempt_path+"attempts_"+mode+"_"+str(vfold)+".txt"

    if (not os.path.exists(self.attempt_path)):
      os.makedirs(self.attempt_path)
      self.init_files()
    elif (not os.path.isfile(self.attempt_file)):
      self.init_files()
    else:
      print(f'attempt is already initialized')
      print(f'Experiment\'s _{mode} attempt no_ : {self.get_attempt()}')
  
  def init_files(self):
    attempt = 1
    f = open(self.attempt_file,'w')
    with open(self.attempt_file, 'a') as out:
      out.write(str(attempt))
      print(f'Experiment\'s attempt no_ : {attempt}')
  
  def add_attempt(self):
    with open(self.attempt_file, "r") as f:
      attempt = f.read()
      attempt = int(attempt)
      attempt+=1
    with open(self.attempt_file, "w") as f:
      f.write(str(attempt))
    print(f'Experiment\'s attempt changed to : {attempt}')

  def get_attempt(self):
    with open(self.attempt_file, "r") as f:
      attempt = int(f.read())
    return attempt

  def set_attempt(self,attempt_value):
    with open(self.attempt_file, "w") as f:
      f.write(str(attempt_value))
    print(f'Experiment\'s attempt is set to : {attempt_value}')

In [ ]:
#train_model_func
def train(model, loss_fn, train_loader, valid_loader,
          epochs, optimizer, learning_rate, device, classes, expid, mode, vfold, modelid):

  exp_attempt = attempt_class(mode, vfold)
  
  print('Train started..')

  train_losses = []
  valid_losses = []
  
  microrecall = []
  macrorecall = []
  
  microprecision = []
  macroprecision = []
  
  microf1 = []
  macrof1 = []
  
  classf1 = []
  
  #console print redirect - save output of train to a log_file
  console_path = "/content/"+expid+mode+'/'+expid+'_'+str(mode)+'_'+str(vfold)+".txt"
  
  epoch_instance = epochs_class()
  total_epochs = epoch_instance.set_total(epochs)
  epoch = epoch_instance.get_step()

  prevent_overfit = prevent_overfitting()
  
  model_name = expid+'_'+mode+'_v'+str(vfold)+'_a'+str(exp_attempt.get_attempt())+'_'+modelid
  
  while (epoch<=total_epochs):

    model.train()
    batch_losses=[]

    for i,data in tqdm(enumerate(train_loader)):
      x, y = data
      optimizer.zero_grad()
      x = x.to(device, dtype=torch.float32)
      y = y.to(device, dtype=torch.long) 
      y_hat = model(x) 
      loss = loss_fn(y_hat, y)
      loss.backward()
      batch_losses.append(loss.item())						
      optimizer.step()
    train_losses.append(batch_losses)
    mean_train_losses=([np.mean(l) for l in train_losses])
    
    print()
    print(f'Epoch - {epoch} Train-Loss : {np.mean(train_losses[-1])}')
    print(f'\nEpoch - {epoch} Train-Loss : {np.mean(train_losses[-1])}\n', file=open(console_path, "a"))
    print()
    
    #Validation step
    model.eval()
     
    batch_losses=[]
    trace_y = []
    trace_yhat = []
    
    for i, data in enumerate(valid_loader):
      x, y = data
      x = x.to(device, dtype=torch.float32)
      y = y.to(device, dtype=torch.long)
      y_hat = model(x)
      loss = loss_fn(y_hat, y)
      trace_y.append(y.cpu().detach().numpy())
      trace_yhat.append(y_hat.cpu().detach().numpy())      
      batch_losses.append(loss.item())
    valid_losses.append(batch_losses)
    mean_valid_losses=([np.mean(l) for l in valid_losses])
    
    trace_y = np.concatenate(trace_y)
    trace_yhat = np.concatenate(trace_yhat)

    #f1,micro,macro
    micro_recall,micro_precision,micro, macro_recall,macro_precision,macro = F1_score(trace_y,trace_yhat,classes)
    
    microrecall.append(micro_recall)
    microprecision.append(micro_precision)
    microf1.append(micro) 

    macrorecall.append(macro_recall)
    macroprecision.append(macro_precision)
    macrof1.append(macro)
    
    #f1 for each class
    f1_class = F1_Class(trace_y,trace_yhat,classes)
    classf1.append(f1_class)

    #console prints
    print(f'Epoch - {epoch} Valid-Loss : {np.mean(valid_losses[-1])}')
    print(f'micro_f1_{epoch} = {micro} ')
    print(f'macro_f1_{epoch} = {macro} ')
    #append log_file
    print(f'Epoch - {epoch} Valid-Loss : {np.mean(valid_losses[-1])}', file=open(console_path, "a"))
    print(f'micro_f1_{epoch} = {micro} ', file=open(console_path, "a"))
    print(f'macro_f1_{epoch} = {macro} ', file=open(console_path, "a"))
    '''
    #check if overfitting and reduce lr twice, after that, in case loss is ascending terminate train    
    early_stop, optimizer = overfit.detect_overfitting(np.mean(valid_losses[-1]),optimizer, learning_rate, epoch_instance)
    if early_stop:
      total_epochs=epoch_instance.set_total(epoch_instance.get_step())
    epoch=epoch_instance.next_step()
    '''
    #check if overfitting and reduce lr twice, after that, in case loss is ascending terminate train    
    early_stop = prevent_overfit.detect_overfitting(np.mean(valid_losses[-1]), epoch, console_path)
    if early_stop:
      total_epochs=prevent_overfit.early_stopping(epoch_instance)
    
    #always store the best according to avg_micro_and_macro_F1
    current_best, best_micro, best_macro = prevent_overfit.store_best_model(micro,macro,console_path)#, best_epoch
    #If achieves current best mean accuracy, Save the model
    if current_best:
      save_model(model_name, vfold, model.state_dict())
      backup_metrics(trace_y,trace_yhat,classes,results_path+'backup/')
    
    #next epoch
    epoch=epoch_instance.next_step()
  
  #calc_trainable_params
  #params=sum(p.numel() for p in model.parameters() if p.requires_grad)
  model_parameters = filter(lambda p: p.requires_grad, model.parameters())
  params = sum([np.prod(p.size()) for p in model_parameters])
      
  #Save analytic results 
  save_results(define_content(total_epochs,
		                          mean_train_losses,
                              mean_valid_losses, 
                              microrecall,
                              microprecision,
                              microf1, 
                              macrorecall,
                              macroprecision,
                              macrof1,
		                          classf1),
                      define_filenames_pattern(expid, mode, vfold, exp_attempt.get_attempt()))
  
      
  #Save general results so as to quickly compare systems
  genresfilename = genres_filename(expid,mode)
  save_genres(round(best_micro,3), round(best_macro,3), params, vfold, genresfilename)
  print('params =', params)
  print('params =', params, file=open(console_path, "a") )
  
  #Save the model
  #model_name = expid+'_'+mode+'_v'+str(vfold)+'_a'+str(exp_attempt.get_attempt())+'_'+modelid
  #save_model(model_name, model.state_dict())

  exp_attempt.add_attempt()
  ZipAnDownload(mode) 
  return  train_losses,valid_losses

In [ ]:
#single fold train
train_losses,valid_losses = train(model, loss_fn, train_loader, valid_loader, epochs, optimizer, learning_rate, device, us8k_classes, expid, mode, vfold, modelid)

In [ ]:
if mode =='80':
  !rm -r us280 us280.zip
elif mode =='128':
  !rm -r us2128 us2128.zip
elif mode =='360':
  !rm -r esc2360 esc2360.zip